In [3]:
# Imports & parameters
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import stats

In [10]:
# Data import
folder = 'ohlcv_binance_perp'
dfs = {}

for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        symbol = filename.replace('.csv', '')
        df = pd.read_csv(os.path.join(folder, filename), parse_dates=['date'] if 'date' in pd.read_csv(os.path.join(folder, filename), nrows=0).columns else [0])
        df = df.set_index(df.columns[0])
        df.sort_index(inplace=True)
        dfs[symbol] = df

In [12]:
dfs['BTCUSDT']

,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
open_time,,,,,,,,,,,
2019-12-31,7240.80,7319.00,7141.00,7189.00,89203.437,2019-12-31 23:59:59.999,6.437921e+08,157014,42880.850,3.095523e+08,0
2020-01-01,7189.43,7260.43,7170.15,7197.57,56801.329,2020-01-01 23:59:59.999,4.096788e+08,101871,28834.200,2.080020e+08,0
2020-01-02,7197.57,7209.59,6922.00,6962.04,115295.677,2020-01-02 23:59:59.999,8.156278e+08,224747,55404.262,3.919117e+08,0
2020-01-03,6962.34,7407.28,6863.44,7341.72,208493.458,2020-01-03 23:59:59.999,1.507314e+09,409820,107485.965,7.770568e+08,0
2020-01-04,7341.60,7400.00,7269.21,7350.71,92586.033,2020-01-04 23:59:59.999,6.784024e+08,186349,46139.090,3.381396e+08,0
...,...,...,...,...,...,...,...,...,...,...,...
2026-02-06,62868.10,71714.40,59800.00,70544.50,559182.769,2026-02-06 23:59:59.999,3.714715e+10,16332904,283473.971,1.886395e+10,0
2026-02-07,70544.50,71690.00,67250.00,69254.20,266663.651,2026-02-07 23:59:59.999,1.843686e+10,8784245,131785.540,9.110083e+09,0
2026-02-08,69254.30,72300.00,68843.70,70288.10,162753.587,2026-02-08 23:59:59.999,1.147750e+10,5758737,84081.188,5.930510e+09,0
